In [112]:
import pandas as pd
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import pickle
loaded_model = pickle.load(open("data/rf_regressor.pkl", 'rb'))

c:\Users\adamk\Projects\angle-of-repose-ML\.venvML\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.1.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\adamk\Projects\angle-of-repose-ML\.venvML\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 1.1.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\adamk\Projects\angle-of-repose-ML\.venvML\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator Pipeline from version 1.1.1 when using v

In [113]:
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

In [114]:
df = pd.read_excel("data\DEM Simulations Data.xlsx", usecols="A:I")
df.head()


,Name,friction coeff,adhesive dist,force fraction,restitution coeff,rolling resistance,tangential stiffness,simulated AoR,predicted AoR
0,sim2,0.44,0.0011,0.22,0.22,0.32,0.52,42.957606,NaN
1,sim4,0.35,0.0007,0.40,0.17,0.32,0.75,43.763328,NaN
2,sim5,0.20,0.0005,0.10,0.10,0.20,0.20,28.576794,NaN
3,sim7,0.80,0.0005,0.34,0.22,0.56,0.20,46.152702,NaN
4,sim8,0.38,0.0013,0.31,0.12,0.51,0.60,51.285944,NaN


In [115]:
drop_indexes = [2, 3, 10, 11, 12, 13, 14, 15, 18]
df_drop = df.copy()
df_drop.drop(drop_indexes, axis=0, inplace=True)
df_drop.head()

,Name,friction coeff,adhesive dist,force fraction,restitution coeff,rolling resistance,tangential stiffness,simulated AoR,predicted AoR
0,sim2,0.44,0.0011,0.22,0.22,0.32,0.52,42.957606,NaN
1,sim4,0.35,0.0007,0.40,0.17,0.32,0.75,43.763328,NaN
4,sim8,0.38,0.0013,0.31,0.12,0.51,0.60,51.285944,NaN
5,sim9,0.50,0.0010,0.20,0.20,0.40,0.70,48.167879,NaN
6,sim10,0.30,0.0012,0.32,0.29,0.30,0.40,46.872942,NaN


In [116]:
X = df.to_numpy()
X_drop = df_drop.to_numpy()

In [117]:
X

array([['sim2', 0.44, 0.0011, 0.22, 0.22, 0.32, 0.52, 42.9576062664117,
        nan],
       ['sim4', 0.35, 0.0007, 0.4, 0.17, 0.32, 0.75, 43.7633284772604,
        nan],
       ['sim5', 0.2, 0.0005, 0.1, 0.1, 0.2, 0.2, 28.5767935646229, nan],
       ['sim7', 0.8, 0.0005, 0.34, 0.22, 0.56, 0.2, 46.1527020049389,
        nan],
       ['sim8', 0.38, 0.0013, 0.31, 0.12, 0.51, 0.6, 51.2859442518568,
        nan],
       ['sim9', 0.5, 0.001, 0.2, 0.2, 0.4, 0.7, 48.1678787092821, nan],
       ['sim10', 0.3, 0.0012, 0.32, 0.29, 0.3, 0.4, 46.8729418163991,
        nan],
       ['sim11', 0.4, 0.0006, 0.22, 0.2, 0.4, 0.2, 39.967677810189, nan],
       ['sim12', 0.49, 0.0012, 0.21, 0.23, 0.24, 0.2, 41.1666628688084,
        nan],
       ['sim14', 0.72, 0.0006, 0.1, 0.1, 0.23, 0.3, 38.2182312645208,
        nan],
       ['sim15', 0.47, 0.0008, 0.36, 0.31, 0.53, 0.6, 51.2859442518568,
        nan],
       ['sim16', 0.25, 0.0015, 0.33, 0.13, 0.22, 0.25, 47.7951146801621,
        nan],
       ['sim17

In [128]:
errors = []
errors_drop = []
predicted_aor = []
predicted_aor_drop = []
for i in X:
    result = loaded_model.predict(i[1:7].reshape(6,-1).T)
    # print(i)
    # print(result)
    diff = i[7]-result
    errors.append(float(diff))
    # print(f"{diff=}")
    predicted_aor.append(float(result))
 
for i in X_drop:
    result = loaded_model.predict(i[1:7].reshape(6,-1).T)
    # print(i)
    # print(result)
    diff = i[7]-result
    errors_drop.append(float(diff))
    # print(f"{diff=}")
    predicted_aor_drop.append(float(result))   

print(predicted_aor)
print(predicted_aor_drop)

[41.419719753989824, 42.56416737661214, 35.621767714646474, 52.288033190883155, 48.68402175867788, 48.53551231660816, 45.689916461715384, 40.14554081209256, 39.634628645751334, 38.179803185397304, 44.17862315532627, 44.865729304428186, 44.302455505766176, 45.971881527777725, 52.00022669304803, 49.75139110633133, 45.24523787617022, 38.57020444444445, 41.1366728968254, 43.322622698412694, 41.817207135431566]
[41.419719753989824, 42.56416737661214, 48.68402175867788, 48.53551231660816, 45.689916461715384, 40.14554081209256, 39.634628645751334, 38.179803185397304, 45.24523787617022, 38.57020444444445, 43.322622698412694, 41.817207135431566]


c:\Users\adamk\Projects\angle-of-repose-ML\.venvML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\adamk\Projects\angle-of-repose-ML\.venvML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\adamk\Projects\angle-of-repose-ML\.venvML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\adamk\Projects\angle-of-repose-ML\.venvML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\adamk\Projects\angle-of-repose-ML\.venvML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but Rand

In [129]:
df['predicted AoR'] = predicted_aor
df['error'] = errors
df_drop['predicted AoR'] = predicted_aor_drop
df_drop['error'] = errors_drop


In [120]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

In [132]:
y_test = np.array(df['simulated AoR'])
y_hat = np.array(predicted_aor)
y_test_dropped = np.array(df_drop['simulated AoR'])
y_hat_dropped = np.array(predicted_aor_drop)
print(y_test.shape)
print(y_hat.shape)
print(y_test_dropped.shape)
print(y_hat_dropped.shape)


(21,)
(21,)
(12,)
(12,)


In [133]:
print(f'{len(y_hat_dropped)=}, {len(y_test_dropped)}')

len(y_hat_dropped)=12, 12


In [134]:
y_test_dropped

array([42.95760627, 43.76332848, 51.28594425, 48.16787871, 46.87294182,
       39.96767781, 41.16666287, 38.21823126, 43.17532701, 40.53209108,
       40.21174985, 40.85040611])

In [135]:
for i in range(len(y_test)):
    print(f'{y_test[i]=:.2f}, {y_test[i]=:.2f}')

y_test[i]=42.96, y_test[i]=42.96
y_test[i]=43.76, y_test[i]=43.76
y_test[i]=28.58, y_test[i]=28.58
y_test[i]=46.15, y_test[i]=46.15
y_test[i]=51.29, y_test[i]=51.29
y_test[i]=48.17, y_test[i]=48.17
y_test[i]=46.87, y_test[i]=46.87
y_test[i]=39.97, y_test[i]=39.97
y_test[i]=41.17, y_test[i]=41.17
y_test[i]=38.22, y_test[i]=38.22
y_test[i]=51.29, y_test[i]=51.29
y_test[i]=47.80, y_test[i]=47.80
y_test[i]=39.06, y_test[i]=39.06
y_test[i]=52.11, y_test[i]=52.11
y_test[i]=50.32, y_test[i]=50.32
y_test[i]=47.54, y_test[i]=47.54
y_test[i]=43.18, y_test[i]=43.18
y_test[i]=40.53, y_test[i]=40.53
y_test[i]=43.58, y_test[i]=43.58
y_test[i]=40.21, y_test[i]=40.21
y_test[i]=40.85, y_test[i]=40.85


In [139]:
pred_r2_drop = r2_score(y_test_dropped, y_hat_dropped) 
pred_MAE_drop = mean_absolute_error(y_test_dropped, y_hat_dropped)
pred_MSE_drop = mean_squared_error(y_test_dropped, y_hat_dropped)

pred_r2 = r2_score(y_test, y_hat) 
pred_MAE = mean_absolute_error(y_test, y_hat)
pred_MSE = mean_squared_error(y_test, y_hat)
print(f"Errors without parameters close to boundaries:\n\t{pred_r2_drop=:.2f}\t{pred_MAE_drop=:.2f}\t{pred_MSE_drop=:.2f}")

print(f"Errors with all simulations:\n\t{pred_r2=:.2f}\t\t{pred_MAE=:.2f}\t\t{pred_MSE=:.2f}")
# es.append([algo, curr_r2, curr_MAE, curr_MSE])

Errors without parameters close to boundaries:
	pred_r2_drop=0.80	pred_MAE_drop=1.40	pred_MSE_drop=2.77
Errors with all simulations:
	pred_r2=0.59		pred_MAE=2.75		pred_MSE=12.31


In [140]:
writer = pd.ExcelWriter('predicted_aor_dropped.xlsx')
# write dataframe to excel
 
df_drop.to_excel(writer)
 
# save the excel
writer.save()
print("DataFrame is exported successfully to 'converted-to-excel.xlsx' Excel File.")


DataFrame is exported successfully to 'converted-to-excel.xlsx' Excel File.


C:\Users\adamk\AppData\Local\Temp\ipykernel_14132\2516569513.py:7: FutureWarning: save is not part of the public API, usage can give in unexpected results and will be removed in a future version
  writer.save()
